In [1]:
import tensorflow as tf
import numpy as np
import scipy
from scipy import misc
import glob
from PIL import Image
import os
import matplotlib.pyplot as plt
import librosa
from keras import layers
from keras.layers import (Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, 
                          Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D)
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.optimizers import Adam
from keras.initializers import glorot_uniform
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from pydub import AudioSegment
import shutil
from keras.preprocessing.image import ImageDataGenerator
import random

/Users/garethflynn/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/garethflynn/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/garethflynn/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/garethflynn/opt/anaconda3/lib/python3.7/site-

In [30]:
import os
data = 'Data'
genres = list(os.listdir(f'{data}/images_original/'))
genres.remove('.DS_Store')
print(genres)

['pop', 'metal', 'disco', 'blues', 'reggae', 'classical', 'rock', 'hiphop', 'country', 'jazz']


In [35]:
directory = f"{data}/images_original/"
for g in genres:
  filenames = os.listdir(os.path.join(directory,f"{g}"))
  random.shuffle(filenames)
  test_files = filenames[0:30]

  for f in test_files:

    #shutil.move(directory + f"{g}"+ "/" + f,f"{data}/images_test/" + f"{g}")


In [49]:
data_gen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)
path='Data/images_original' 
image_datagentrain=data_gen.flow_from_directory(path,target_size=(288,432),color_mode="rgba",batch_size=1,class_mode='categorical')

Found 699 images belonging to 10 classes.


In [50]:
path='Data/images_test'
image_datagentest=data_gen.flow_from_directory(path,target_size=(288,432),color_mode="rgba",batch_size=1,class_mode='categorical')

Found 300 images belonging to 10 classes.


In [51]:
def GenreModel(input_shape = (288,432,4),classes=10):
  np.random.seed(10)
  X_input = Input(input_shape)

  X = Conv2D(8,kernel_size=(3,3),strides=(1,1),kernel_initializer = glorot_uniform(seed=9))(X_input)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)
  
  X = Conv2D(16,kernel_size=(3,3),strides = (1,1),kernel_initializer=glorot_uniform(seed=9))(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)
  
  X = Conv2D(32,kernel_size=(3,3),strides = (1,1),kernel_initializer = glorot_uniform(seed=9))(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)

  X = Conv2D(64,kernel_size=(3,3),strides=(1,1),kernel_initializer=glorot_uniform(seed=9))(X)
  X = BatchNormalization(axis=-1)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)

  
  X = Flatten()(X)

  X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=9))(X)

  model = Model(inputs=X_input,outputs=X,name='GenreModel')

  return model

In [56]:
import keras.backend as K
def get_f1(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val
  
model = GenreModel(input_shape=(288,432,4),classes=10)
opt = Adam(learning_rate=0.0005)
model.compile(optimizer = opt,loss='categorical_crossentropy',metrics=['accuracy',get_f1]) 

model.fit_generator(image_datagentrain, epochs=10, validation_data = image_datagentest)

Epoch 1/70
699/699 [==============================] - 114s 163ms/step - loss: 4.2162 - accuracy: 0.2790 - get_f1: 0.2275 - val_loss: 0.0905 - val_accuracy: 0.2100 - val_get_f1: 0.1500
Epoch 2/70
699/699 [==============================] - 144s 206ms/step - loss: 1.9081 - accuracy: 0.4249 - get_f1: 0.3090 - val_loss: 0.2289 - val_accuracy: 0.3267 - val_get_f1: 0.2867
Epoch 3/70
699/699 [==============================] - 142s 203ms/step - loss: 1.2021 - accuracy: 0.5894 - get_f1: 0.5050 - val_loss: 1.4154 - val_accuracy: 0.4467 - val_get_f1: 0.3767
Epoch 4/70
699/699 [==============================] - 187s 267ms/step - loss: 0.8222 - accuracy: 0.7253 - get_f1: 0.6581 - val_loss: 3.0952 - val_accuracy: 0.4200 - val_get_f1: 0.3600
Epoch 5/70
699/699 [==============================] - 139s 199ms/step - loss: 0.5363 - accuracy: 0.8340 - get_f1: 0.7940 - val_loss: 0.5035 - val_accuracy: 0.4900 - val_get_f1: 0.4400
Epoch 6/70
699/699 [==============================] - 115s 165ms/step - loss: 0.

699/699 [==============================] - 82s 118ms/step - loss: 0.0123 - accuracy: 0.9971 - get_f1: 0.9971 - val_loss: 3.7535 - val_accuracy: 0.5533 - val_get_f1: 0.5367
Epoch 46/70
699/699 [==============================] - 82s 117ms/step - loss: 0.0055 - accuracy: 0.9986 - get_f1: 0.9986 - val_loss: 0.3061 - val_accuracy: 0.5633 - val_get_f1: 0.5533
Epoch 47/70
699/699 [==============================] - 82s 118ms/step - loss: 0.0058 - accuracy: 0.9986 - get_f1: 0.9986 - val_loss: 0.0184 - val_accuracy: 0.5600 - val_get_f1: 0.5233
Epoch 48/70
699/699 [==============================] - 82s 118ms/step - loss: 0.0072 - accuracy: 0.9971 - get_f1: 0.9971 - val_loss: 0.6949 - val_accuracy: 0.5433 - val_get_f1: 0.5300
Epoch 49/70
699/699 [==============================] - 82s 117ms/step - loss: 0.0102 - accuracy: 0.9971 - get_f1: 0.9971 - val_loss: 0.0061 - val_accuracy: 0.5800 - val_get_f1: 0.5633
Epoch 50/70
699/699 [==============================] - 82s 117ms/step - loss: 0.0113 - accur